<a href="https://colab.research.google.com/github/Agil0509/NLP/blob/main/gpt_dev_main(S%C9%99m%C9%99d_Vur%C4%9Fun).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
unwanted_chars = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9','\n']

with open('/content/Semed_Vurgun_seirleri.txt', 'r', encoding='utf-8') as file:
    text = file.read()

for char in unwanted_chars:
    text = text.replace(char, '  ')

with open('Semed_Vurgun_cleaned_file.txt', 'w', encoding='utf-8') as file:
    file.write(text)

# print("Data cleaned and saved to 'cleaned_file.txt'")


In [ ]:
text[:1000]

'  CAVANLARA XİTAB  Mənim düşkün könlüm sizdən eləyir  Bu fərz*   olan təmənnanı, cavanlar.  Unutmayın bəşər sizdən diləyır,  Elm, ədəb, həm irfani, cavanlar.  Bəşər gülşəninin bülbülüsünüz,  Şanlı firqəmizin sağ əlisiniz,  Zalım əllilərin cəlladısınız,  Dəvasızların dərmanı, cavanlar.  Carəsız dərdlərə çarə etməyə,  Quşu vurub həm imdada yetməyə,  Həyatı ağ, nə kı qarə etməyə  Sərf ediniz hər zamanı, cavanlar.  Yaşayarsa bəşər, yaşar sizinlə,  Soğulmuş  †   çeşmələr axar sizinlə,  Zalımlar geridən baxar, sizinlə  İnqilabın qəhrəmanı, cavanlar.  Çünki tələb edir zavallı bəşər,  Gərək keçə sizdən ona bir əsər.  Yoxsa qan ağlayar yenə sərbəsər,  Olmaz da dərdinə çarə, cavanlar.  İstəsəniz, ölkəmiz, abad olar,  Ellərin məhbusları azad olar,  Onda da Vurğunun könlü şad olar,  Verər sizə can qurbanı, cavanlar.                *Vacib, zəruri  †Qurumuş, tutulmuş             MAYDAN  ...Haman sənə məhəl-məmuriyyətim*   olan Köçəsgər kəndindən         rübləyə bir maydan at almşam. Bu qiymətə həqi

In [ ]:
len(text)

872234

In [ ]:
len(''.join(set(text)))

100

In [ ]:
set(text)

{'\x01',
 ' ',
 '!',
 '"',
 "'",
 '(',
 ')',
 '*',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '>',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'X',
 'Y',
 'Z',
 '[',
 '\\',
 ']',
 '^',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '§',
 '°',
 '»',
 'Ç',
 'Ö',
 'Ü',
 'ä',
 'ç',
 'ö',
 'ü',
 'Ğ',
 'ğ',
 'İ',
 'ı',
 'Ş',
 'ş',
 'Ə',
 'ə',
 '–',
 '—',
 '‘',
 '’',
 '“',
 '”',
 '†',
 '‡',
 '…',
 '★'}

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

batch_size = 64
block_size = 256
max_iters = 1500
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2
torch.manual_seed(42)



chars = sorted(list(set(text)))
vocab_size = len(chars)


stoi = {}
for i,ch in enumerate(chars):
  stoi[ch] = i

itos = {}
for i,ch in enumerate(chars):
  itos[i] = ch

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

import torch

data = torch.tensor(encode(text),dtype = torch.long)

n= int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]


def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data)- block_size,(batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  x, y = x.to(device), y.to(device)
  return x,y

@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()
  for split in ['train','val']:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X,Y = get_batch(split)
      logits, loss = model(X,Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out

class Head(nn.Module):
  def __init__(self,head_size):
    super().__init__()
    self.key = nn.Linear(n_embd,head_size,bias = False)
    self.query = nn.Linear(n_embd,head_size,bias = False)
    self.value = nn.Linear(n_embd,head_size,bias = False)
    self.register_buffer('tril', torch.tril(torch.ones(block_size,block_size)))

  def forward(self,x):
    B, T,C = x.shape
    k = self.key(x)
    q = self.query(x)

    wei = q @ k.transpose(-2,-1) * C**-0.5
    wei = wei.masked_fill(self.tril[:T,: T] == 0, float('-inf'))
    wei = F.softmax(wei, dim = -1)
    v = self.value(x)
    out = wei  @ v
    return out




class MultiHeadAttention(nn.Module):
  def __init__(self, num_heads, head_size):
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.proj = nn.Linear(n_embd,n_embd)

  def forward(self,x):
    out = torch.cat([h(x) for h in self.heads],dim =-1)
    out = self.proj(out)
    return out


class FeedForward(nn.Module):
  def __init__(self,n_embd):
    super().__init__()
    self.net = nn.Sequential(
      nn.Linear(n_embd,4* n_embd),
      nn.ReLU(),
      nn.Linear(4* n_embd, n_embd),
      nn.Dropout(dropout)
    )
  def forward(self,x):
    return self.net(x)


class Block(nn.Module):
  def __init__(self, n_embd,n_head):
    super().__init__()
    head_size = n_embd // n_head
    self.sa = MultiHeadAttention(n_head,head_size)
    self.ffwd = FeedForward(n_embd)
    self.ln1 = nn.LayerNorm(n_embd)
    self.ln2 = nn.LayerNorm(n_embd)

  def forward(self,x):
    x = x+ self.sa(self.ln1(x))
    x = x + self.ffwd(self.ln2(x))
    return x


class GPTLanguageModel(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
    self.position_embedding_table = nn.Embedding(block_size, n_embd)

    self.blocks = nn.Sequential(*[Block(n_embd, n_head = n_head) for _ in range(n_layer)])
    self.ln_f = nn.LayerNorm(n_embd)
    # self.sa_head = Head(n_embd)
    # self.sa_heads = MultiHeadAttention(4,n_embd//4)
    # self.ffwd = FeedForward(n_embd)

    # self.blocks = nn.Sequential(
    #   Block(n_embd, n_head = 4),
    #   Block(n_embd, n_head = 4),
    #   Block(n_embd, n_head = 4),
    #   nn.LayerNorm(n_embd)
    # )

    self.lm_head = nn.Linear(n_embd,vocab_size)

  def forward(self, idx, targets=None):

    B, T= idx.shape
    token_emb = self.token_embedding_table(idx)
    pos_emb = self.position_embedding_table(torch.arange(T, device=device))
    x = token_emb+pos_emb

    # x = self.sa_head(x)
    # x = self.sa_heads(x)
    # x = self.ffwd(x)
    x = self.blocks(x)
    logits = self.lm_head(x)

    if targets is None:
      loss = None

    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)    #B - batch_size , T - time_step , C - Chanel
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
      for _ in range(max_new_tokens):
        idx_cond = idx[:,-block_size:]
        logits,loss = self(idx_cond)

        # logits, loss = self(idx)
        logits = logits[:, -1, :]
        probs = F.softmax(logits, dim = -1)
        idx_next = torch.multinomial(probs, num_samples = 1)
        idx = torch.cat((idx, idx_next), dim = 1)   #cat = concat
      return idx

model = GPTLanguageModel(vocab_size)
m = model.to(device)

print(sum(p.numel() for p in m.parameters()) / 1e6, 'M parameters')

optimizer = torch.optim.Adam(m.parameters(), lr= 1e-3)

for iter in range(max_iters):
  if iter % eval_iters == 0:
    losses = estimate_loss()
    print(f'step {iter}: loss {losses}')

  xb, yb = get_batch('train')
  logits, loss = model(xb,yb)
  optimizer.zero_grad(set_to_none = True)
  loss.backward()
  optimizer.step()

context = idx = torch.zeros((1, 1), dtype = torch.long, device = device)

print(decode(m.generate(context, max_new_tokens = 1000)[0].tolist()))






device cuda
10.815844 M parameters
step 0: loss {'train': tensor(5.0154), 'val': tensor(5.0197)}
step 200: loss {'train': tensor(2.4865), 'val': tensor(2.4760)}
step 400: loss {'train': tensor(1.9778), 'val': tensor(1.9645)}
step 600: loss {'train': tensor(1.7539), 'val': tensor(1.7532)}
step 800: loss {'train': tensor(1.5998), 'val': tensor(1.6430)}
step 1000: loss {'train': tensor(1.4830), 'val': tensor(1.5811)}
step 1200: loss {'train': tensor(1.3882), 'val': tensor(1.5446)}
step 1400: loss {'train': tensor(1.2924), 'val': tensor(1.5430)}
niyanın uynuzəhərdən.  Sadə o gözəl və dövrəmizarı  Oturmuşdur başında mənalı bir dən,  Utanmadın öz görəmizə!  Yaman, içdi dünya köçəcəəyəm mən...  Biz eyvanı da qəlbimdə sülmisən?  İnsan gəimizdir niyət - arabən,  Doqqussa, dünya bizim Bəxtiyar,  Bizim Sakit o, qaşlar dilbəti  Xasın, bizim gözlərin də bircəsi  Öpərin ilk ömür yadigar...  Ucadır buğ sardaca ki,  Atılır zimzənət axır düzü  Zəhərlənəci nəğməsin ürək...  Tüstüləcək gözlə gülümsüz ya

In [ ]:
context = idx = torch.zeros((1, 1), dtype = torch.long, device = device)

print(decode(m.generate(context, max_new_tokens = 1000)[0].tolist()))

əşmə kimi, öz rəzzi baş kimi  Çəkib sözlə suram meyləşənlər.  O can var çoxmuşamış sinəsillər;  Osağ səhərin zamanlar, suları –  Qaya gəlir, yenə də atır salır;  Dəyişmiş bulaqların kimi salır –  Üzəndən içəndə həqi o taləmi  Sən, əlbətdir zəfərələr...  Əli, Ölmüş bir qəsəndə  Yenə ürək sarı oturan sayıqda;  Nə od içim? Tökərin var, nə torpağa  Mətən, yenə köynət ilhama aparmaq?  Dedim ki, son gözəl meyas?  ardım:  Gözəllər görünən guzludur...  Neçin bunlar  Səninlə qanad alır,  Keçim də böyük ürəyi.  Aşıq qalib kelib,  hələ kən qayıtmadım büz gülən.  Dedim kimi kamalınma ürək qüyərdə,  Ölünc ilimdəki qaldımızmı?  Çəndəmi dərdli, bir yandırsan,  Unun girməsinidən qaldırar  Kənabın qayğusu gələcəkdir!  Beşidir əzəldən, sən də!  Elə bil gündə bəlkə bir günəşdə.  Tən təzən şörək yeyib alib kəsə  Qoy Sən bizim BAYRamanım Nələrin?   **  , ona, bəlkə həməni bəxtəriyəm  Deyən Cavab düzdü yatan,  Seyə bir vaxtımın öz qəlbində  Həyata salıb yamr eləm, heçlərim!  İndidə mərin, qüvvət yellərim v